In [52]:
import pandas as pd

In [61]:
%run createDerivation.py

In [70]:
def image_to_vector(image: np.ndarray, final_length: int = 2048, max_features_to_detect: int = 5000) -> np.ndarray | None:
    """
    Converts an image into a fixed-length 1D feature vector using ORB.

    The function detects ORB features, selects the strongest ones, and flattens
    their descriptors into a single vector.

    Args:
        image (np.ndarray): The input image (can be color or grayscale).
        final_length (int): The desired length of the output vector.
                              MUST be a multiple of 32. Defaults to 1600 (50 features * 32).
        max_features_to_detect (int): The maximum number of features for ORB to detect initially.
                                      Should be significantly larger than (final_length / 32).

    Returns:
        np.ndarray | None: A 1D NumPy array of dtype uint8 with the specified length,
                          or None if not enough features could be detected in the image.
    """
    # 1. Validate inputs
    if image is None:
        print("Error: Input image is None.")
        return None

    if final_length % 32 != 0:
        raise ValueError("Error: final_length must be a multiple of 32.")

    # Calculate the exact number of features we need to sample
    num_features_to_sample = final_length // 32

    # 2. Ensure the image is grayscale
    if len(image.shape) == 3 and image.shape[2] == 3:
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray_image = image # Assume it's already grayscale

    # 3. Initialize ORB detector
    orb = cv2.ORB_create(nfeatures=max_features_to_detect)

    # 4. Detect keypoints and compute descriptors
    kps, des = orb.detectAndCompute(gray_image, None)

    # 5. Check if enough descriptors were found
    if des is None or len(des) < num_features_to_sample:
        print(f"Warning: Found only {0 if des is None else len(des)} features, but need {num_features_to_sample}. Cannot create a vector of length {final_length}.")
        return None

    # 6. Sort features by response score (strongest first)
    kp_des_pairs = sorted(zip(kps, des), key=lambda x: x[0].response, reverse=True)

    # 7. Select the top N features
    top_pairs = kp_des_pairs[:num_features_to_sample]
    
    # We only need the descriptors from the top pairs
    _, sampled_des = zip(*top_pairs)
    
    # 8. Flatten the list of descriptors into a single 1D vector
    feature_vector = np.array(sampled_des).flatten()

    return feature_vector
def keep_all(path): return True
def path_2_features(path: str) -> str | None:
    """
    Reads an image from the given path and converts it to a string representation of its feature vector.
    in comma-separated format.
    Args:
        path (str): The file path to the image.
    Returns:
        str | None: A comma-separated string of the feature vector, or None if conversion fails
    """ 
    try:
        img = cv2.imread(path)
        if img is None:
            print(f"Error: Unable to read image at path: {path}")
            return None
        feature_vector = image_to_vector(img)
        if feature_vector is None:
            return "0"
        return ','.join(map(str, feature_vector.tolist())).encode('utf-8')
    except Exception as e:
        print(f"Exception occurred while processing image at path {path}: {e}")
        return None


In [71]:
file_content = ""
with open("final_variation.txt", 'r') as file:
    file_content = file.read()

filtered2_path = file_content.strip()
filtered2_df = pd.read_csv(filtered2_path)

In [74]:
processed = create_dataset_variation(filtered2_df, keep_all, path_2_features, variation_tag="orb_features_2048")
print("Created variation CSV with features:", processed)
extracted_features_path = open("final_features.txt", "w")
extracted_features_path.write(processed)
extracted_features_path.close()

Wrote 416 rows to /home/fadhlan/Normal2/DeepLearningRepo/steps/variations/var_ec55b06b1b0972c8/ec55b06b1b0972c8.csv (variation ec55b06b1b0972c8)
Created variation CSV with features: /home/fadhlan/Normal2/DeepLearningRepo/steps/variations/var_ec55b06b1b0972c8/ec55b06b1b0972c8.csv
